In [1]:
from sdm.utils import set_project_wd
set_project_wd()

Current Working Directory: /Users/matthewwhittle/Data Science/shefflied-bats


In [2]:
import ee

# Initialize Earth Engine
ee.Initialize()


In [3]:
import geopandas as gpd
boundary = gpd.read_file('data/processed/boundary.geojson')
# Add a 10km buffer to the boundary
boundary["geometry"] = boundary.buffer(20_000)
boundary

,County,geometry
0,South Yorkshire,"POLYGON ((392859.732 393429.711, 392697.300 39..."


In [4]:
# Get the lat lon bounds of the AOI
boundary = boundary.to_crs(4326)
bbox = boundary.bounds.values[0]
print(bbox)
xmin, ymin, xmax, ymax = bbox

[-2.1388582  53.11308682 -0.54825054 53.84981282]


# Get a sentinel-2 image

Lidar data was captured during 2022 so we will get an image in 2022

In [5]:
# PArams
# Summer 2022 will give the most accurate representation of vegetation for the lidar capture period
START_DATE  = "2022-06-01"
END_DATE    = "2022-08-31"

In [6]:
x = boundary.geometry.centroid.x.values[0]
y = boundary.geometry.centroid.y.values[0]


/var/folders/vm/9ws15vws62zgnr64rdr_ls5r0000gn/T/ipykernel_60127/3755768224.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x = boundary.geometry.centroid.x.values[0]
/var/folders/vm/9ws15vws62zgnr64rdr_ls5r0000gn/T/ipykernel_60127/3755768224.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y = boundary.geometry.centroid.y.values[0]


In [7]:
AOI = ee.Geometry.Point(x,y)
START_DATE = '2022-05-01'
END_DATE = '2022-10-30'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [8]:
from re import A
from sdm.data.sentinel import Sentinel2CloudMasker
AOI = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])
masker = Sentinel2CloudMasker(
    aoi=AOI,
    start_date=START_DATE,
    end_date=END_DATE,
    cloud_filter=CLOUD_FILTER,
    cld_prb_thresh=CLD_PRB_THRESH,
    nir_drk_thresh=NIR_DRK_THRESH,
    cld_prj_dist=CLD_PRJ_DIST,
    buffer=BUFFER,
    )

cloud_masked_image = masker.get_cloud_masked_image()


In [9]:
import geemap
# Visualize (This part is for demonstration and won't run in a standalone Python script)
Map = geemap.Map()
Map.centerObject(AOI, 10)
Map.addLayer(cloud_masked_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1}, 'RGB')
Map


Map(center=[53.48302334723679, -1.3435543682535172], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
google_drive_folder = "earth_engine"
file_prefix = "sentinel2_cloud_free_south_yorkshire_2022"

aoi_box = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])
# Export as TIFF to Google Drive
task = ee.batch.Export.image.toDrive(
    image=cloud_masked_image,
    description='Sentinel2_CloudFree',
#    folder=google_drive_folder,  # specify the folder name here
#    fileNamePrefix=file_prefix,  # specify the file name prefix here
    scale=10,
    region=aoi_box,
    crs='EPSG:27700',
    fileFormat='GeoTIFF'
)
task.start()

print("Export task started!")

Export task started!


In [11]:
import time
task.status()
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(5)

Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id: B4FLA7ERYE25K6AYUISKRB75).
Polling for task (id